# Experiment 2 Units vs dropout

Import data

In [10]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
import gin

Hoe maak je automatisch een file model.gin?

In [11]:
gin.parse_config_file("model.gin")

/home/azureuser/MachineLearning/.venv/lib/python3.11/site-packages/gin/config.py:615: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  decorated_class = decorating_meta(cls.__name__, (cls,), overrides)


ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

In [12]:
preprocessor = BasePreprocessor()
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2024-12-10 17:37:26.357 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2024-12-10 17:37:26.358 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [13]:
print(gin.config_str())

import gin.torch.external_configurables

# Parameters for NeuralNetwork:
# ==============================================================================
NeuralNetwork.num_classes = 10
NeuralNetwork.units1 = 512



A big advantage is that we can save this config as a file; that way it is easy to track what you changed during your experiments.

In [14]:
accuracy = metrics.Accuracy()

Or, you can use gin, it will read the model.gin file, and instead of needing to set

You can gin.parce_config_file('model.gin') and then set the model with model = NeuralNetwork(), and the parameters will be loaded from the gin file.

If you want to combine this with a manual grid search, you could automate that with a double forloop:

In [15]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 128
Units: 128, 64
Units: 64, 64


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

!!! Train_steps en valid_steps is verkleind om de runtime op de vm werkbaar te houden. Dit is gezet op 64. Dus 64 batches.

!!!! Epochs vergroot naar 50 om te zien wanneer er een kantelpunt is. Wanneer ga je overfitten?

!!!! Uit experiment 1 bleek Adam of AdamW beter te werken dan SGD. Daarom optimizer ingesteld op Adam

In [ ]:
import torch
gin.parse_config_file("model.gin")

units = [256, 128, 64]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=50,
    metrics=[accuracy],
    logdir=(f"Dropout_0.1_{unit1}_{unit2}"),	
    train_steps=128,
    valid_steps=128,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit1 in units:
    for unit2 in units:
        if unit1 < unit2:
            continue
        gin.bind_parameter("NeuralNetwork.units1", unit1)
        gin.bind_parameter("NeuralNetwork.units2", unit2)

        settings.logdir = f"Dropout_0.1_{unit1}_{unit2}"

        model = imagemodels.NeuralNetwork()
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )

        trainer.loop()

2024-12-08 14:00:13.263 | INFO     | mltrainer.settings:check_path:61 - Created logdir /home/azureuser/MachineLearning/notebooks/1_pytorch_intro/SGD_64_64
2024-12-08 14:00:13.268 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to SGD_256_256/20241208-140013
2024-12-08 14:00:13.270 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 128/128 [00:00<00:00, 239.21it/s]
2024-12-08 14:00:14.181 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 2.2965 test 2.2883 metric ['0.1730']
100%|██████████| 128/128 [00:00<00:00, 257.49it/s]
2024-12-08 14:00:15.038 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 2.2815 test 2.2736 metric ['0.2184']
100%|██████████| 128/128 [00:00<00:00, 254.53it/s]
2024-12-08 14:00:15.928 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 2.2664 test 2.2583 metric ['0.2247']
100%|██████████| 128/128 [00:00<00:00, 242.65it/s]
2024-12-08 14:00:16

In [16]:
settings

NameError: name 'settings' is not defined

In [79]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

!!! Er zijn 10 verschillende fashion categorien. Dus out_features = 10

In [ ]:
settings

In [ ]:
model

Run the experiment, and study the result with tensorboard.

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

in the terminal, cd to the location of the repository
activate the python environment for the shell. Note how the correct environment is being activated.
run tensorboard --logdir=modellogs in the terminal
tensorboard will launch at localhost:6006 and vscode will notify you that the port is forwarded
you can either press the launch button in VScode or open your local browser at localhost:6006

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`


Experiment with things like:

- changing the amount of units1 and units2 to values between 16 and 1024. Use factors of 2: 16, 32, 64, etc.
- changing the batchsize to values between 4 and 128. Again, use factors of two.
- all your experiments are saved in the `modellogs` directory, with a timestamp. Inside you find a saved_config.gin file, that 
contains all the settings for that experiment. The `events` file is what tensorboard will show.
- plot the result in a heatmap: units vs batchsize.
- changing the learningrate to values between 1e-2 and 1e-5 
- changing the optimizer from SGD to one of the other available algoritms at [torch](https://pytorch.org/docs/stable/optim.html) (scroll down for the algorithms)

A note on train_steps: this is a setting that determines how often you get an update. 
Because our complete dataset is 938 (60000 / 64) batches long, you will need 938 trainstep to cover the complete 60.000 images.

This can actually be a bit confusion, because every value below 938 changes the meaning of `epoch` slightly, because one epoch is no longer
the full dataset, but simply `trainstep` batches. Setting trainsteps to 100 means you need to wait twice as long before you get feedback on the performance,
as compared to trainsteps=50. You will also see that settings trainsteps to 100 improves the learning, but that is simply because the model has seen twice as 
much examples as compared to trainsteps=50.

This implies that it is not usefull to compare trainsteps=50 and trainsteps=100, because setting it to 100 will always be better.
Just pick an amount, and adjust your number of epochs accordingly.